In [1]:
import os
import glob
import argparse
import easydict
from PIL import Image

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torchvision.models as models
import torchvision.transforms as transforms
from efficientnet_pytorch import EfficientNet

In [2]:
### Arguments
args = easydict.EasyDict({
    "root_test": "/storage0/bxc/data/test_dataset/leaderboard",
    "source_file": "sample_submission.csv",
    #"arch": "resnet18",
    "arch": "efficientnet-b6",
    "resume": "model_best.pth.tar",
    "gpu": 0,
})

In [3]:
model_names = sorted(name for name in models.__dict__
                     if name.islower() and not name.startswith("__")
                     and callable(models.__dict__[name]))

In [4]:
global args, best_acc1

if args.gpu is not None:
    print("Use GPU: {} for inference".format(args.gpu))

# create submission csv file paralle with sample_submission.csv
dir = os.path.dirname(args.source_file)
save_csv = os.path.join(dir, 'efficientnet_b6_1119_oneForEachDir_adamw_epoch4.csv')

sc = open(save_csv, 'w')

sc.write('path,y')
sc.write('\n')

# load model
print("=> creating model '{}'".format(args.arch))
# for i in models.__dict__:
#     print(i)
model = EfficientNet.from_name(args.arch, num_classes=2)

#model = EfficientNet.from_pretrained("efficientnet-b6")
#device = torch.device('cpu')
# modify the number of output nodes
# model.fc = nn.ReLU(True)
# model.fc = nn.Dropout(0.5)
# model.fc = nn.Linear(1000, 512)
# model.fc = nn.ReLU(True)
# model.fc = nn.Dropout(0.2)
# model.fc = nn.Linear(512, 2)

model = model.cuda(args.gpu)


# load pretrained weight
if args.resume:
    if os.path.isfile(args.resume):
        print("=> loading checkpoint '{}'".format(args.resume))
        if args.gpu is None:
            checkpoint = torch.load(args.resume)
        else:
            # Map model to be loaded to specified single gpu.
            loc = 'cuda:{}'.format(args.gpu)
            checkpoint = torch.load(args.resume, map_location=loc)
        args.start_epoch = checkpoint['epoch']
        best_acc1 = checkpoint['best_acc1']
        if args.gpu is not None:
            # best_acc1 may be from a checkpoint from a different GPU
            best_acc1 = best_acc1.to(args.gpu)
        model.load_state_dict(checkpoint['state_dict'])
        print("=> loaded checkpoint '{}' (epoch {})"
              .format(args.resume, checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(args.resume))

cudnn.benchmark = True

model = model.eval()

# data loading code
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

transform = transforms.Compose([
    #transforms.CenterCrop(224),
    transforms.Resize(32),
    transforms.ToTensor(),
    normalize,
])

# collect all images
images = glob.glob(os.path.join(args.root_test, '*.jpg'))
images.sort()

# predict label
m = nn.Softmax(dim=1)

with torch.no_grad():
    for image_path in images:
        print(image_path)
        image = Image.open(image_path)
        image = transform(image)
        image = torch.unsqueeze(image, dim=0)

        image = image.cuda(args.gpu, non_blocking=True)
        
        output = model(image)
        #print(output)
        output = m(output)[0]  # apply softmax

        image_tmpl = os.path.join('leaderboard', os.path.basename(image_path))

        # write to submission file
        if output[0] > output[1]:
            saveline = image_tmpl + ',0'
            sc.write(saveline)
            sc.write('\n')
        else:
            saveline = image_tmpl + ',1'
            sc.write(saveline)
            sc.write('\n')

sc.close()

Use GPU: 0 for inference
=> creating model 'efficientnet-b6'
=> loading checkpoint 'model_best.pth.tar'
=> loaded checkpoint 'model_best.pth.tar' (epoch 1)
/storage0/bxc/data/test_dataset/leaderboard/image_00000.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00001.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00002.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00003.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00004.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00005.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00006.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00007.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00008.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00009.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00010.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00011.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00012.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00013.jpg
/sto

/storage0/bxc/data/test_dataset/leaderboard/image_00137.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00138.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00139.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00140.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00141.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00142.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00143.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00144.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00145.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00146.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00147.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00148.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00149.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00150.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00151.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00152.jpg
/storage0/bxc/data/test_dataset/leaderbo

/storage0/bxc/data/test_dataset/leaderboard/image_00281.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00282.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00283.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00284.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00285.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00286.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00287.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00288.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00289.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00290.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00291.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00292.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00293.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00294.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00295.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00296.jpg
/storage0/bxc/data/test_dataset/leaderbo

/storage0/bxc/data/test_dataset/leaderboard/image_00425.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00426.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00427.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00428.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00429.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00430.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00431.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00432.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00433.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00434.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00435.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00436.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00437.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00438.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00439.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00440.jpg
/storage0/bxc/data/test_dataset/leaderbo

/storage0/bxc/data/test_dataset/leaderboard/image_00569.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00570.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00571.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00572.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00573.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00574.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00575.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00576.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00577.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00578.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00579.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00580.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00581.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00582.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00583.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_00584.jpg
/storage0/bxc/data/test_dataset/leaderbo

...

/storage0/bxc/data/test_dataset/leaderboard/image_03449.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03450.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03451.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03452.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03453.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03454.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03455.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03456.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03457.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03458.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03459.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03460.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03461.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03462.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03463.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03464.jpg
/storage0/bxc/data/test_dataset/leaderbo

/storage0/bxc/data/test_dataset/leaderboard/image_03593.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03594.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03595.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03596.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03597.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03598.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03599.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03600.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03601.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03602.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03603.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03604.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03605.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03606.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03607.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03608.jpg
/storage0/bxc/data/test_dataset/leaderbo

/storage0/bxc/data/test_dataset/leaderboard/image_03737.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03738.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03739.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03740.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03741.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03742.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03743.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03744.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03745.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03746.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03747.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03748.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03749.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03750.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03751.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03752.jpg
/storage0/bxc/data/test_dataset/leaderbo

/storage0/bxc/data/test_dataset/leaderboard/image_03880.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03881.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03882.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03883.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03884.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03885.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03886.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03887.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03888.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03889.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03890.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03891.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03892.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03893.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03894.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_03895.jpg
/storage0/bxc/data/test_dataset/leaderbo

/storage0/bxc/data/test_dataset/leaderboard/image_04024.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_04025.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_04026.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_04027.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_04028.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_04029.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_04030.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_04031.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_04032.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_04033.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_04034.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_04035.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_04036.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_04037.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_04038.jpg
/storage0/bxc/data/test_dataset/leaderboard/image_04039.jpg
/storage0/bxc/data/test_dataset/leaderbo